In [1]:
#################################################################
#####      Auteurs : A. Benki, R.Reynier et C. Corbin       #####
#################################################################


"""
Ce script nous permet de faire une comparaison entre les algorithmes 
Machine learning ( KNN, SVM et RF) --->  ( les meilleurs paramètres obtenus par Validation Croisée )

Les résultats obtenus seront l'objet d'une comparaison avec ceux obtenus avec le script:

(script_modelisation_DL_base_Reduite_NANs_Deleted) de Deep learning (RN dense)

Cas D'étude 2 : Base de données réduite 
Traitement NAN : Suppression
"""

print ("-----------------------------------------------------------------------------")
print ("-----------------------Import des packages  ...------------------------------")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
sns.set_theme()

from sklearn import model_selection
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors, metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import VotingClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score



print ("-----------------------------------------------------------------------------")
print ("--------------------------lire la data --------------------------------------")

df=pd.read_csv("../data/weatherAUS.csv")


print ("-----------------------------------------------------------------------------")
print ("# --------- (Var catégorielles vs Var quantitatives) -------------#")

# je recupére les noms des variables numériques du dataframe
result_numer = df.select_dtypes(include='number')

# je recupére les noms des variables catégorielle du dataframe (exclude numerique)
result_categ = df.select_dtypes(exclude='number')




print ("-----------------------------------------------------------------------------")
print ("# --------- (Supprimer les lignes contenant Nan) -------------#")

df = df.dropna()   # 154 k ---> 56 k

print ("-----------------------------------------------------------------------------")
print ("# --------- (traitement Var catégorielles ----> quantitatives) -------------#")

# Traitement colonne date , transformation en 3 colonne jr , mois et année
##########################################################################

df['Date'] = pd.DataFrame(pd.to_datetime(df['Date']))
df.insert(0, "Year", df['Date'].dt.year, allow_duplicates=False)
df.insert(0, "Month", df['Date'].dt.month, allow_duplicates=False)
df.insert(0, "Day", df['Date'].dt.day, allow_duplicates=False)

df = df.drop("Date",1)

# Traitement colonne location ,  affecter à chaque location une valeur numérique
###############################################################################""


hh = df['Location'].value_counts()
hh_df= pd.DataFrame(hh)

for k in range(hh_df.shape[0]) :
    df['Location']=df['Location'].replace(hh_df.index[k],k+1)
    
    
# Traitement colonnes directions Vents location ,  affecter à chaque direction une valeur numérique
###############################################################################################""

#vent = df['WindGustDir'].value_counts()

V_D=['E','N','S','W', 'NE' , 'NW' , 'SE'  , 'SW'  ,'ENE', 'ESE', 'NNE', 'NNW','SSE','SSW','WNW','WSW']
V_D_Num=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

df['WindGustDir'] = df['WindGustDir'].replace(V_D,V_D_Num)
df['WindDir9am'] = df['WindDir9am'].replace(V_D,V_D_Num)
df['WindDir3pm'] = df['WindDir3pm'].replace(V_D,V_D_Num)


# Traitement colonnes Rain_Today , Rain_Tomorrow ,  affecter une valeur numérique (0 si nn, 1 si oui)
####################################################################################################

#pluie = df['RainToday'].value_counts()

Rain_N_Y=['No','Yes']
Rain_N_Y_Num=[0,1]

df['RainToday'] = df['RainToday'].replace(Rain_N_Y,Rain_N_Y_Num)
df['RainTomorrow'] = df['RainTomorrow'].replace(Rain_N_Y,Rain_N_Y_Num)

print ("-----------------------------------------------------------------------------")
print ("# ---- (Verification no NANs et tous les valeurs sont Numériques) ----------#")

# Verification no NANs et tous les valeurs sont Numériques
#########################################################""
res_num_inter = df.select_dtypes(exclude='number')
print(res_num_inter.columns)
df.head()
df.isna().sum()


print ("-----------------------------------------------------------------------------")
print ("# ---- (Copier DataFrame & modelisation ) ----------#")

df_modelisation = df
df_modelisation.head()

df_X=df_modelisation.drop('RainTomorrow',1)
Target=df_modelisation['RainTomorrow']


# split 

X_train, X_test, y_train, y_test = train_test_split(df_X, Target, test_size=0.2, random_state=42)

#Centrer et réduire les variables explicatives des deux échantillons de manière adéquate.

# On applique  la fonction preprocessing.scale
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)


# Dans la suite nous allons comparer plusieurs méthodes d'apprentissage. Pour chacune d'elles, 
# il conviendra d'explorer le périmètre des hyperparamètres suivant :

#-------------------------------------------------------------------------------------------------
from sklearn import neighbors, metrics

#------------KNN ------------

# Les valeurs des hyperparamètres à tester
param_KNN = {'n_neighbors':range(2,41)}

# Création d'un classifieur kNN avec recherche d'hyperparamètre par validation croisée

clf_KNN = model_selection.GridSearchCV(
    neighbors.KNeighborsClassifier(), # un classifieur kNN
    param_KNN,                            # hyperparamètres à tester
    cv=5,                             # nombre de folds de validation croisée
    scoring='accuracy'                # score à optimiser (proportion de prédictions correctes)
)

# Optimiser ce classifieur sur le jeu d'entraînement
clf_KNN.fit(X_train_scaled, y_train)


# Afficher l'hyperparamètre optimal retenu
print("Meilleur hyperparamètre sur le jeu d'entraînement est :")
print(clf_KNN.best_params_)



# Appliquez le modèle à l'ensemble de test:

y_pred_KNN = clf_KNN.predict(X_test_scaled)

# matrice de confusion :

pd.crosstab(y_test, y_pred_KNN, rownames=['Classe réelle'], colnames=['Classe prédite'])


# le score du modèle :

print("le score du modèle KNN est : " , metrics.accuracy_score(y_test, y_pred_KNN))



#------------SVM ------------

# Les valeurs des hyperparamètres à tester

param_SVM = {'C':[0.1, 1], 'kernel':['linear', 'rbf']}


# Création d'un classifieur SVM avec recherche d'hyperparamètre par validation croisée

clf_SVM = model_selection.GridSearchCV(
    svm.SVC(),                        # classification par SVM
    param_SVM,                        # hyperparamètres à tester
    cv=5,                             # nombre de folds de validation croisée
    scoring='accuracy'                # score à optimiser (proportion de prédictions correctes)
)

# Optimiser ce classifieur sur le jeu d'entraînement
clf_SVM.fit(X_train_scaled, y_train)


# Afficher les hyperparamètres optimaux retenus
print("Meilleurs hyperparamètres sur le jeu d'entraînement sont :")
print(clf_SVM.best_params_)

# Appliquez le modèle à l'ensemble de test:

y_pred_SVM = clf_SVM.predict(X_test_scaled)

# matrice de confusion :

pd.crosstab(y_test, y_pred_SVM, rownames=['Classe réelle'], colnames=['Classe prédite'])

# le score du modèle :

print("le score du modèle SVM est : " , metrics.accuracy_score(y_test, y_pred_SVM))



#-------------------------------------------------------------------------------------------------

#------------Random Forest ------------

# Les valeurs des hyperparamètres à tester

param_RF= {'max_features': ["sqrt", "log2", None], 'min_samples_split': range(2, 31,2)}


# Création d'un classifieur RF avec recherche d'hyperparamètre par validation croisée

clf_RF = model_selection.GridSearchCV(
    ensemble.RandomForestClassifier(),   # classification par RF
    param_RF,                           # hyperparamètres à tester
    cv=5,                                # nombre de folds de validation croisée
    scoring='accuracy'                   # score à optimiser (proportion de prédictions correctes)
)

# Optimiser ce classifieur sur le jeu d'entraînement
clf_RF.fit(X_train_scaled, y_train)

# Afficher les hyperparamètres optimaux retenus
print("Meilleurs hyperparamètres sur le jeu d'entraînement sont :")
print(clf_RF.best_params_)

# Appliquez le modèle à l'ensemble de test:

y_pred_RF = clf_RF.predict(X_test_scaled)

# matrice de confusion :

pd.crosstab(y_test, y_pred_RF, rownames=['Classe réelle'], colnames=['Classe prédite'])

# le score du modèle :

print("le score du modèle RF est : " , metrics.accuracy_score(y_test, y_pred_RF))













    
         


-----------------------------------------------------------------------------
-----------------------# Statistique / Nan/ ...------------------------------
-----------------------------------------------------------------------------
# --------- (Var catégorielles vs Var quantitatives) -------------#
-----------------------------------------------------------------------------
# --------- (Supprimer les lignes contenant Nan) -------------#
-----------------------------------------------------------------------------
# --------- (traitement Var catégorielles ----> quantitatives) -------------#


C:\Users\JanaEline\AppData\Local\Temp\ipykernel_7128\1145551993.py:54: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("Date",1)


-----------------------------------------------------------------------------
# ---- (Verification no NANs et tous les valeurs sont Numériques) ----------#
Index([], dtype='object')
-----------------------------------------------------------------------------
# ---- (Copier DataFrame & modelisation ) ----------#


C:\Users\JanaEline\AppData\Local\Temp\ipykernel_7128\1145551993.py:108: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_X=df_modelisation.drop('RainTomorrow',1)
C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typic

C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set 

...

C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set 

C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set 

Meilleur hyperparamètre sur le jeu d'entraînement est :
{'n_neighbors': 39}


C:\Users\JanaEline\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


le score du modèle KNN est :  0.8451790145338532
Meilleurs hyperparamètres sur le jeu d'entraînement sont :
{'C': 1, 'kernel': 'rbf'}
le score du modèle SVM est :  0.858738036157391
Meilleurs hyperparamètres sur le jeu d'entraînement sont :
{'max_features': 'sqrt', 'min_samples_split': 4}
le score du modèle RF est :  0.8611308046791918


In [7]:
print('------------KNN ------------ \n')


print(pd.crosstab(y_test, y_pred_KNN, rownames=['Classe réelle'], colnames=['Classe prédite']))

print('------------SVM ------------ \n')

print(pd.crosstab(y_test, y_pred_SVM, rownames=['Classe réelle'], colnames=['Classe prédite']))

print('------------R F ------------ \n')

print(pd.crosstab(y_test, y_pred_RF, rownames=['Classe réelle'], colnames=['Classe prédite']))

------------KNN ------------ 

Classe prédite     0     1
Classe réelle             
0               8435   364
1               1383  1102
------------SVM ------------ 

Classe prédite     0     1
Classe réelle             
0               8435   364
1               1230  1255
------------R F ------------ 

Classe prédite     0     1
Classe réelle             
0               8411   388
1               1179  1306
